In [ ]:
# Hibakezelés, tesztelés

# Python Course Material: Error Handling and Testing

## Table of Contents

1. Introduction to Error Handling
    - Understanding Exceptions
    - Built-in Exceptions
    - Raising Exceptions
2. Handling Exceptions
    - The `try` Statement
    - The `except` Block
    - Multiple `except` Blocks
    - The `else` Block
    - The `finally` Block
3. Custom Exceptions
    - Creating Custom Exceptions
    - Using Custom Exceptions
4. Testing in Python
    - Introduction to Testing
    - Types of Tests
    - Unit Testing with `unittest`
    - Test Discovery
    - Writing Effective Tests
5. Advanced Testing Techniques
    - Mocking
    - Parameterized Testing
    - Test Coverage
6. Summary and Best Practices

## 1. Introduction to Error Handling

### Understanding Exceptions

In Python, an error is an issue that occurs during the execution of a program. Errors can be syntax errors or exceptions. Exceptions are errors that occur at runtime and can be handled to prevent the program from crashing.

In [1]:
# Example of a runtime error (exception)
print(10 / 0)  # This will raise a ZeroDivisionError

ZeroDivisionError: division by zero

### Built-in Exceptions

Python provides many built-in exceptions, such as `ZeroDivisionError`, `TypeError`, `ValueError`, `KeyError`, and [more](https://docs.python.org/3/library/exceptions.html#exception-hierarchy).

In [3]:
# Examples of built-in exceptions
int('a')  # Raises ValueError

ValueError: invalid literal for int() with base 10: 'a'

In [4]:
[1, 2, 3][5]  # Raises IndexError

IndexError: list index out of range

### Raising Exceptions

You can raise exceptions using the `raise` statement.

In [7]:
# Example of raising an exception
raise ValueError("This is a custom error message")

print("This code does not run")

ValueError: This is a custom error message

## 2. Handling Exceptions

### The `try` Statement

To handle exceptions, use the `try` statement to wrap the code that might raise an exception.

In [8]:
try:
    # Code that might raise an exception
    result = 10 / 0
    print("This code does not run")
except ZeroDivisionError:
    print("Cannot divide by zero")

print("This code runs")

Cannot divide by zero
This code runs


### The `except` Block

The `except` block catches and handles the exception.

In [9]:
try:
    result = int('a')
except ValueError:
    print("Cannot convert string to integer")

Cannot convert string to integer


### Multiple `except` Blocks

You can use multiple `except` blocks to handle different exceptions.

In [10]:
try:
    result = int('a')
except ValueError:
    print("ValueError: Cannot convert string to integer")
except TypeError:
    print("TypeError: Invalid type")

ValueError: Cannot convert string to integer


In some situations, you may want to print or log the exception without actually halting execution. To save the exact error message along with the stack trace, you can use Python's `traceback` module.

The [traceback](https://docs.python.org/3/library/traceback.html) module in Python provides a way to extract, format, and print stack traces of Python programs. This is particularly useful when you're dealing with exceptions and want to get more detailed information about where an error occurred.

In [9]:
import traceback

def risky_function():
    # This function will intentionally raise a ZeroDivisionError
    return 1 / 0

def main():
    try:
        risky_function()
    except ZeroDivisionError as e:
        print(traceback.format_exc())

main()


Traceback (most recent call last):
  File "C:\Users\gregk\AppData\Local\Temp\ipykernel_21320\1764533044.py", line 9, in main
    risky_function()
  File "C:\Users\gregk\AppData\Local\Temp\ipykernel_21320\1764533044.py", line 5, in risky_function
    return 1 / 0
           ~~^~~
ZeroDivisionError: division by zero



### Re-Raising Exceptions as Different Exceptions

Sometimes, you may need to catch an exception, perform some actions (e.g., logging or additional processing), and then re-raise it as a different type of exception. This can be useful for abstracting away implementation details and providing a more meaningful error context.

#### Example: Re-Raising Exceptions

In [16]:
class CustomError(Exception):
    """A custom exception type."""
    pass

class AnotherError(Exception):
    """Another custom exception type."""
    pass

def process_data(data):
    if data == "bad":
        raise ValueError("An error occurred with the data")

def main():
    try:
        process_data("bad")
    except ValueError as e:
        # Log the original exception (could also perform other actions)
        print(f"Logging original error: {e}")
        # Re-raise a different exception with a custom message
        raise CustomError("A custom error occurred during processing") from e

try:
    main()
except CustomError as e:
    print(f"Handled custom error: {e}")
    print(f"Original exception: {e.__cause__}")

Logging original error: An error occurred with the data
Handled custom error: A custom error occurred during processing
Original exception: An error occurred with the data


**Explanation:**

- `process_data` raises a `ValueError`.
- In the `main` function, the `ValueError` is caught, and a `CustomError` is raised instead, using the `raise ... from e` syntax to retain the context of the original exception.
- `e.__cause__` provides access to the original `ValueError`.

### Re-Raising the same Exception

You can also re-raise an exception in the `except` block:

In [14]:
def divide_numbers(a, b):
    try:
        result = a / b
        return result
    except ZeroDivisionError as e:
        # Handle the ZeroDivisionError
        print(f"Caught an exception: {e}")
        # Re-raise the same exception
        raise

result = divide_numbers(10, 0)

Caught an exception: division by zero


ZeroDivisionError: division by zero

### Exception Chaining

When re-raising exceptions, Python allows you to chain exceptions to maintain the context of the original exception. This is done using the `raise ... from ...` syntax, as shown in the previous example. 

#### Example: Exception Chaining

In [15]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError as e:
        # Handle or log the original exception
        print(f"Logging original ZeroDivisionError: {e}")
        # Re-raise a new exception with additional context
        raise ValueError("Failed to perform division") from e

try:
    result = divide(10, 0)
except ValueError as e:
    print(f"Handled ValueError: {e}")
    print(f"Original exception: {e.__cause__}")

Logging original ZeroDivisionError: division by zero
Handled ValueError: Failed to perform division
Original exception: division by zero


**Explanation:**

- `divide` catches `ZeroDivisionError` and re-raises it as a `ValueError`.
- The `ValueError` retains the context of the `ZeroDivisionError`, which can be accessed through `e.__cause__`.

### The `else` Block

The `else` block is executed if no exceptions are raised in the `try` block.

In [12]:
try:
    result = int('10')
except ValueError:
    print("Cannot convert string to integer")
else:
    print("Conversion successful:", result)

Conversion successful: 10


Practical example:

In [13]:
try:
    # Try to open and read the file
    with open('myfile.txt', 'r') as file:
        content = file.read()
except FileNotFoundError:
    # Handle the case where the file does not exist
    print("Error: The file was not found.")
except IOError:
    # Handle other I/O errors
    print("Error: An I/O error occurred.")
else:
    # This block will execute if no exceptions were raised
    # Process the file content
    print("File content successfully read. Processing content...")
    # Example processing: count the number of lines in the file
    num_lines = len(content.splitlines())
    print(f"The file has {num_lines} lines.")

# Output message indicating the end of the script
print("End of script.")


File content successfully read. Processing content...
The file has 3 lines.
End of script.


What would have happened if we tried to access the `content` variable when the file was not found?

---

On a related note, see the following (off-topic) example for the `else` block in `for` loops:

In [14]:
numbers = [1, 3, 5, 7, 9, 11, 13]
target = 8

# Loop through the list to find the target number
for number in numbers:
    if number == target:
        print(f"Found the target number: {target}")
        break
else:
    # This block will execute if the loop completes without encountering a break
    print(f"Target number {target} not found in the list.")

# Output message indicating the end of the script
print("End of search.")

Target number 8 not found in the list.
End of search.


This also works with `while` loops.

### The `finally` Block

The `finally` block is always executed, regardless of whether an exception was raised or not. This is particularly useful for resource cleanup tasks such as closing files or releasing locks.

In [15]:
try:
    result = 10 / 0
except ZeroDivisionError:
    print("Cannot divide by zero")
finally:
    print("This will always execute")

Cannot divide by zero
This will always execute


A more practical example:

In [16]:
def read_file(file_path):
    try:
        # Try to open and read the file
        file = open(file_path, 'r')
        content = file.read()
        print("File content successfully read.")
        return content
    except FileNotFoundError:
        # Handle the case where the file does not exist
        print(f"Error: The file at {file_path} was not found.")
    except Exception as e:
        # Handle any other exceptions that may occur
        print(f"An error occurred: {e}")
    finally:
        # This block will always execute, regardless of whether an exception was raised or not
        try:
            file.close()
            print("File closed.")
        except NameError:
            # Handle the case where 'file' was never opened
            print("File was never opened, so it cannot be closed.")
        except Exception as e:
            # Handle any other exceptions that may occur during file closing
            print(f"An error occurred while closing the file: {e}")

# Example usage
file_path = "myfile.txt"
content = read_file(file_path)


File content successfully read.
File closed.


**Note on file handling using context managers**

Python provides a more concise and reliable way to handle files using the `with` statement, which simplifies file handling and ensures that files are properly closed without needing explicit exception handling for closing the file. This method automatically takes care of closing the file even if an error occurs.

Here’s how you can use the `with` statement for file handling:

In [17]:
def read_file(file_path):
    try:
        # Using 'with' statement to open and read the file
        with open(file_path, 'r') as file:
            content = file.read()
            print("File content successfully read.")
            return content
    except FileNotFoundError:
        # Handle the case where the file does not exist
        print(f"Error: The file at {file_path} was not found.")
    except Exception as e:
        # Handle any other exceptions that may occur
        print(f"An error occurred: {e}")

# Example usage
file_path = "example.txt"
content = read_file(file_path)


Error: The file at example.txt was not found.


## 3. Custom Exceptions

### Creating Custom Exceptions

You can create custom exceptions by defining a new class that inherits from the `Exception` class.

In [23]:
class CustomError(Exception):
    pass

# Raising a custom exception
raise CustomError("This is a custom error")

CustomError: This is a custom error

### Using Custom Exceptions

Custom exceptions can be used just like built-in exceptions.

In [24]:
try:
    raise CustomError("This is a custom error")
except CustomError as e:
    print(e)

This is a custom error


## 4. Testing in Python

### Introduction to Testing

Testing is the process of evaluating and verifying that a software program or application does what it is supposed to do.

### Types of Tests

- Unit Tests: Test individual components or functions.
- Integration Tests: Test how different components work together.
- System Tests: Test the complete system.
- Acceptance Tests: Test if the system meets business requirements.

### Unit Testing with [unittest](https://docs.python.org/3.6/library/unittest.html)

Python's `unittest` module provides a framework for creating and running tests.

In [37]:
import unittest

def add(a, b):
    return a + b

class TestMath(unittest.TestCase):
    def test_add(self):
        self.assertEqual(add(1, 2), 3)
        self.assertEqual(add(-1, 1), 0)

"""
# For runing in VSCode or non-jupyter environment
if __name__ == '__main__':
    unittest.main()
"""

unittest.main(argv=[''], verbosity=2, exit=False)

test_add (__main__.TestMath.test_add) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


The list of assert methods in unittest is documented [here](https://docs.python.org/3.6/library/unittest.html#assert-methods).

### Test Discovery

`unittest` can automatically discover tests in a directory.

```sh
python -m unittest discover
```

### Unit Testing with [doctest](https://docs.python.org/3.6/library/doctest.html)

Python's `doctest` module provides a framework for creating and running tests.

In [38]:
import doctest

def add(a, b):
    '''
    This is a test:
    >>> add(2, 2)
    5

    This is a second test:
    >>> add(1, 2)
    3
    '''
    return a + b

doctest.testmod(verbose=True)

Trying:
    add(2, 2)
Expecting:
    5
**********************************************************************
File "__main__", line 6, in __main__.add
Failed example:
    add(2, 2)
Expected:
    5
Got:
    4
Trying:
    add(1, 2)
Expecting:
    3
ok
9 items had no tests:
    __main__
    __main__.CustomError
    __main__.CustomResource
    __main__.CustomResource.__enter__
    __main__.CustomResource.__exit__
    __main__.TestMath
    __main__.TestMath.test_add
    __main__.custom_context
    __main__.read_file
**********************************************************************
1 items had failures:
   1 of   2 in __main__.add
2 tests in 10 items.
1 passed and 1 failed.
***Test Failed*** 1 failures.


TestResults(failed=1, attempted=2)

## Exercise: Factorial calculation with doctests

### Instructions:

1. **Implement the `factorial` function**: Write the `factorial` function so that it computes the factorial of a non-negative integer `n`. The function should also raise a `ValueError` if a negative integer is provided.

2. **Include Tests**: Add appropriate test cases in the docstring of the function. Ensure that the test cases cover different scenarios including edge cases.

3. **Run `doctest`**: Use the `doctest` framework to run the tests. Ensure that all tests pass and check the output.

### Example code:

In [9]:
import doctest

# Implement the factorial function, include tests for 0, 1, 5 and 3 input values.
def factorial(n):
    # ...

# Run the doctests
doctest.testmod(verbose=True)

Trying:
    factorial(0)
Expecting:
    1
ok
Trying:
    factorial(1)
Expecting:
    1
ok
Trying:
    factorial(5)
Expecting:
    120
ok
Trying:
    factorial(3)
Expecting:
    6
ok
1 items had no tests:
    __main__
1 items passed all tests:
   4 tests in __main__.factorial
4 tests in 2 items.
4 passed and 0 failed.
Test passed.


TestResults(failed=0, attempted=4)

In [10]:
# Solution

import doctest

def factorial(n):
    '''
    Returns the factorial of a non-negative integer n.
    
    A factorial of a number n (n!) is the product of all positive integers less than or equal to n.
    
    For example:
    >>> factorial(0)
    1
    >>> factorial(1)
    1
    >>> factorial(5)
    120
    >>> factorial(3)
    6
    
    Raises:
    ValueError: If n is a negative integer.
    '''
    if n < 0:
        raise ValueError("Input must be a non-negative integer.")
    if n == 0:
        return 1
    return n * factorial(n - 1)

doctest.testmod(verbose=True)

Trying:
    factorial(0)
Expecting:
    1
ok
Trying:
    factorial(1)
Expecting:
    1
ok
Trying:
    factorial(5)
Expecting:
    120
ok
Trying:
    factorial(3)
Expecting:
    6
ok
1 items had no tests:
    __main__
1 items passed all tests:
   4 tests in __main__.factorial
4 tests in 2 items.
4 passed and 0 failed.
Test passed.


TestResults(failed=0, attempted=4)

### Writing Effective Tests

Effective tests are:
- Independent and repeatable
- Comprehensive and cover edge cases
- Easy to read and maintain

## 5. Advanced Testing Techniques

### Mocking

Mocking is used to replace parts of the system under test and make assertions about how they are used.

In [41]:
import unittest
from unittest.mock import Mock

# Assume we have a service class that interacts with an external API
class ExternalAPIService:
    def fetch_data(self):
        # Imagine this method makes a network call to fetch data
        raise NotImplementedError("This should be implemented by a real API call.")

# A client class that uses the ExternalAPIService
class DataProcessor:
    def __init__(self, api_service):
        self.api_service = api_service

    def process(self):
        data = self.api_service.fetch_data()
        # Some processing logic here
        return data * 2

# Unit test case using unittest framework
class TestDataProcessor(unittest.TestCase):
    def test_process(self):
        # Create a mock for the ExternalAPIService
        mock_api_service = Mock(spec=ExternalAPIService)
        mock_api_service.fetch_data.return_value = 10

        # Inject the mock into the DataProcessor
        processor = DataProcessor(mock_api_service)

        # Run the method under test
        result = processor.process()

        # Assert that the result is as expected
        self.assertEqual(result, 20)

        # Verify that fetch_data was called once
        mock_api_service.fetch_data.assert_called_once()

unittest.main(argv=[''], verbosity=2, exit=False)

test_process (__main__.TestDataProcessor.test_process) ... ok
test_add (__main__.TestMath.test_add) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.008s

OK


### Parameterized Testing

Parameterized testing allows running the same test with different inputs. To see an example of parametrized testing, we have to install the [parametrized](https://github.com/wolever/parameterized) module.

In [43]:
!pip install parameterized


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [47]:
from parameterized import parameterized
import unittest

def add(a, b):
    return a + b

class TestMath(unittest.TestCase):
    @parameterized.expand([
        (1, 2, 3),
        (-1, 1, 0),
        (0, 0, 0),
    ])
    def test_add(self, a, b, expected):
        self.assertEqual(add(a, b), expected)

unittest.main(argv=[''], verbosity=2, exit=False)

test_process (__main__.TestDataProcessor.test_process) ... ok
test_add_0 (__main__.TestMath.test_add_0) ... ok
test_add_1 (__main__.TestMath.test_add_1) ... ok
test_add_2 (__main__.TestMath.test_add_2) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.008s

OK


### Test Coverage

Test coverage measures how much of your code is tested by your test suite. Tools like `coverage.py` can be used to measure this.

```sh
coverage run -m unittest discover
coverage report
```

In [3]:
!pip install coverage

   ---------------------------------------- 0.0/209.5 kB ? eta -:--:--
   - -------------------------------------- 10.2/209.5 kB ? eta -:--:--
   ----- --------------------------------- 30.7/209.5 kB 445.2 kB/s eta 0:00:01
   ----------- --------------------------- 61.4/209.5 kB 550.5 kB/s eta 0:00:01
   ---------------------------------------  204.8/209.5 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 209.5/209.5 kB 1.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# Run tests with coverage measurement
!coverage run -m unittest discover -v

test_deposit (banking_system.tests.test_account.TestAccount.test_deposit) ... ok
test_deposit_negative_amount (banking_system.tests.test_account.TestAccount.test_deposit_negative_amount) ... ok
test_withdraw (banking_system.tests.test_account.TestAccount.test_withdraw) ... ok
test_withdraw_insufficient_funds (banking_system.tests.test_account.TestAccount.test_withdraw_insufficient_funds) ... ok
test_create_account (banking_system.tests.test_bank.TestBank.test_create_account) ... ok
test_create_account_existing (banking_system.tests.test_bank.TestBank.test_create_account_existing) ... ok
test_process_invalid_transaction (banking_system.tests.test_bank.TestBank.test_process_invalid_transaction) ... ok
test_process_transaction (banking_system.tests.test_bank.TestBank.test_process_transaction) ... ok
test_execute_deposit (banking_system.tests.test_transaction.TestTransaction.test_execute_deposit) ... ok
test_execute_invalid_transaction_type (banking_system.tests.test_transaction.TestTransa

In [6]:
# Generate coverage report
!coverage report -m

Name                                       Stmts   Miss  Cover   Missing
------------------------------------------------------------------------
banking_system\__init__.py                     0      0   100%
banking_system\account.py                     16      1    94%   13
banking_system\bank.py                        17      1    94%   19
banking_system\tests\__init__.py               0      0   100%
banking_system\tests\test_account.py          19      1    95%   25
banking_system\tests\test_bank.py             21      1    95%   27
banking_system\tests\test_transaction.py      20      1    95%   25
banking_system\transaction.py                 11      0   100%
------------------------------------------------------------------------
TOTAL                                        104      5    95%


In [7]:
# Generate HTML coverage report (optional)
!coverage html

Wrote HTML report to htmlcov\index.html


## 6. Summary and Best Practices

### Best Practices for Error Handling

- Use exceptions for exceptional conditions.
- Catch specific exceptions instead of a generic `except` block.
- Avoid using exceptions for control flow.
- Clean up resources in the `finally` block.

### Best Practices for Testing

- Write tests for all new features and bug fixes.
- Keep tests simple and focused on one thing.
- Use descriptive names for test methods.
- Run tests frequently to catch issues early.


*Note:* You can use a broad exception handler, but it's generally not recommended:

```python
try:
    do_something()
except:
    print("Caught an exception!")
```

This approach will catch all exceptions, including those you might not want to handle, like `KeyboardInterrupt`. To avoid this, it's better to catch specific exceptions or re-raise the exception after logging it. Here's an improved example:

```python
try:
    do_something()
except IOError as e:
    print(f"I/O error({e.errno}): {e.strerror}")
except ValueError:
    print("Could not convert data to an integer.")
except Exception as e:
    print("Unexpected error:", type(e).__name__)
    raise
```

In this example, `IOError` and `ValueError` are handled separately, while any other unexpected exceptions are caught and re-raised after printing the error type.

In [8]:
import time

# Imagine the situation if the outer loop was an infinite loop
for i in range(3):
    try:
        for i in range(10):
            time.sleep(1)
    except:
        print("Caught an exception!")


Caught an exception!
Caught an exception!
Caught an exception!
Caught an exception!


---

# Further notes regarding context managers

The `with` block in Python uses context managers to manage resources. A context manager is an object that defines the runtime context to be established when executing a `with` statement. Context managers typically use the `__enter__` and `__exit__` methods to set up and tear down the context.

You can create your own context managers by either implementing the `__enter__` and `__exit__` methods in a class or by using the `contextlib` module, which provides a decorator for creating context managers.

### Creating a Context Manager by Implementing `__enter__` and `__exit__`

Here's an example of creating a custom context manager by implementing the `__enter__` and `__exit__` methods in a class:

In [19]:
class CustomResource:
    def __enter__(self):
        print("Entering the context")
        # Initialization code here
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        print("Exiting the context")
        # Cleanup code here
        if exc_type:
            print(f"An exception occurred: {exc_value}")
        # Returning False re-raises the exception, True suppresses it
        return False

# Using the custom context manager
with CustomResource() as resource:
    print("Inside the context")
    # Perform operations within the context

print("Outside the context")

Entering the context
Inside the context
Exiting the context
Outside the context


### Creating a Context Manager Using `contextlib` Module

The `contextlib` module provides a convenient way to create context managers using the `@contextmanager` decorator:

In [21]:
from contextlib import contextmanager

@contextmanager
def custom_context():
    print("Entering the context")
    # Initialization code here
    try:
        yield
        # Code inside the 'with' block runs here
    finally:
        print("Exiting the context")
        # Cleanup code here

# Using the custom context manager
with custom_context():
    print("Inside the context")
    # Perform operations within the context

print("Outside the context")

Entering the context
Inside the context
Exiting the context
Outside the context


### Explanation

1. **Custom Class with `__enter__` and `__exit__`**:
   - The `__enter__` method is executed when the `with` block is entered.
   - The `__exit__` method is executed when the `with` block is exited.
   - Any exceptions raised inside the `with` block are passed to the `__exit__` method.

2. **Using `contextlib.contextmanager` Decorator**:
   - The function decorated with `@contextmanager` becomes a context manager.
   - The code before the `yield` statement is executed when the `with` block is entered.
   - The code after the `yield` statement is executed when the `with` block is exited.
   - The `yield` statement can pass a value to the variable after `as` in the `with` statement.

Both methods allow you to create custom context managers that can manage resources and ensure proper setup and teardown, making your code more robust and readable.

# Sources

 - [Python built-in Exception hierarchy](https://docs.python.org/3/library/exceptions.html#exception-hierarchy)
 - [Traceback module documentation](https://docs.python.org/3/library/traceback.html)
 - [Python unittest framework](https://docs.python.org/3.6/library/unittest.html)
 - [Python unittest framework - list of assertion methods](https://docs.python.org/3.6/library/unittest.html#assert-methods)
 - [doctest library](https://docs.python.org/3.6/library/doctest.html)
 - [parametrized library](https://github.com/wolever/parameterized)